In [1]:
import os
import subprocess
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.ops import unary_union

In [2]:
# 📂 Rutas
ruta_csv = r"C:/Proyectos/2024/Conejos/mapas/pozos.csv"
ruta_salida = r"C:/Proyectos/2024/Conejos/mapas/interpolados"
os.makedirs(ruta_salida, exist_ok=True)

In [3]:
os.environ["PATH"] += r";C:\OSGeo4W\bin"
subprocess.run(["gdal_grid", "--version"])

CompletedProcess(args=['gdal_grid', '--version'], returncode=0)

In [4]:
# 📌 Ruta del ejecutable de gdal_grid
gdal_grid_exe = r"C:\OSGeo4W\bin\gdal_grid.exe"

In [5]:
# 📌 Configuración
campo_x = "x_nad27"
campo_y = "y_nad27"
años_validos = [2005, 2006, 2007, 2009, 2010, 2011, 2012, 2015, 2016, 2017, 2018] ##

In [6]:
# 📥 Leer CSV con Pandas
df = pd.read_csv(ruta_csv)

In [7]:
# 1) Límite en 26713
gdf_limite = gpd.read_file('C:/Proyectos/2024/Conejos/mapas/aol_conejos.shp').to_crs(epsg=26713)
print ("\U0001F40D")
# Verificamos proyecciones
print("CRS límite:", gdf_limite.crs)

🐍
CRS límite: EPSG:26713


In [15]:
minx, miny, maxx, maxy = gdf_limite.total_bounds
print("Bounds con buffer:", minx, miny, maxx, maxy)  # <-- verifica que cambian

Bounds con buffer: 281035.2291087191 3408589.12309631 361971.7835115908 3518366.062258497


In [21]:
# 2) Buffer de 2 km (ajusta a tu gusto)
buffer_m = 2000
# 3) Bounds del buffer
minx, miny, maxx, maxy = gdf_limite.buffer(buffer_m).total_bounds
print("Bounds con buffer:", minx, miny, maxx, maxy)  # <-- verifica que cambian
res = 500  # metros/píxel


print ("\U0001F40D")

Bounds con buffer: 279036.3497009068 3406589.2008453473 363970.6403670179 3520365.6544953054
🐍


In [22]:
for anio in años_validos:
    campo_valor = str(anio)

    datos = df[
        df["x_nad27"].notna() &
        df["y_nad27"].notna() &
        df[campo_valor].notna()
    ][["x_nad27", "y_nad27", campo_valor]]

    if len(datos) < 3:
        print(f"⚠️ Solo {len(datos)} valores válidos para {campo_valor}, se omite.")
        continue

    # Renombrar la columna de valores para que GDAL la acepte
    col_z = f"val_{campo_valor}"
    datos = datos.rename(columns={campo_valor: col_z})

    # # Calcular límites
    # min_x, max_x = datos["x_nad27"].min(), datos["x_nad27"].max()
    # min_y, max_y = datos["y_nad27"].min(), datos["y_nad27"].max()

    # Guardar CSV temporal limpio
    temp_csv = os.path.join(ruta_salida, f"temp_{campo_valor}.csv")
    datos.to_csv(temp_csv, index=False)

    salida_raster = os.path.join(ruta_salida, f"nivel_{campo_valor}.tif")

    # Comando GDAL
    comando = [
        "gdal_grid",
        # "-overwrite",  # asegura que reemplace el tif
        "-a", "invdist:power=2.0:smoothing=0.0",
        "-txe", str(minx), str(maxx),
        "-tye", str(miny), str(maxy),  # <-- ymin, ymax
        "-tr", str(res), str(res),       # resolución fija en 500 m
        "-zfield", col_z,
        "-oo", "X_POSSIBLE_NAMES=x_nad27",
        "-oo", "Y_POSSIBLE_NAMES=y_nad27",
        "-a_srs", "+proj=utm +zone=13 +datum=NAD27 +units=m +no_defs",  # 👈 aquí  
        "-of", "GTiff",
        f"CSV:{temp_csv}",
        salida_raster
    ]

    print(f"🔹 Ejecutando: {' '.join(comando)}")
    result = subprocess.run(comando, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"⚠️ Error con {campo_valor}")
        print("STDOUT:", result.stdout.strip())
        print("STDERR:", result.stderr.strip())
    else:
        print(f"✅ Guardado: {salida_raster}")


🔹 Ejecutando: gdal_grid -a invdist:power=2.0:smoothing=0.0 -txe 279036.3497009068 363970.6403670179 -tye 3406589.2008453473 3520365.6544953054 -tr 500 500 -zfield val_2005 -oo X_POSSIBLE_NAMES=x_nad27 -oo Y_POSSIBLE_NAMES=y_nad27 -a_srs +proj=utm +zone=13 +datum=NAD27 +units=m +no_defs -of GTiff CSV:C:/Proyectos/2024/Conejos/mapas/interpolados\temp_2005.csv C:/Proyectos/2024/Conejos/mapas/interpolados\nivel_2005.tif
✅ Guardado: C:/Proyectos/2024/Conejos/mapas/interpolados\nivel_2005.tif
🔹 Ejecutando: gdal_grid -a invdist:power=2.0:smoothing=0.0 -txe 279036.3497009068 363970.6403670179 -tye 3406589.2008453473 3520365.6544953054 -tr 500 500 -zfield val_2006 -oo X_POSSIBLE_NAMES=x_nad27 -oo Y_POSSIBLE_NAMES=y_nad27 -a_srs +proj=utm +zone=13 +datum=NAD27 +units=m +no_defs -of GTiff CSV:C:/Proyectos/2024/Conejos/mapas/interpolados\temp_2006.csv C:/Proyectos/2024/Conejos/mapas/interpolados\nivel_2006.tif
✅ Guardado: C:/Proyectos/2024/Conejos/mapas/interpolados\nivel_2006.tif
🔹 Ejecutando: gd